In [5]:
# extract each column and its value from html tables
import os
import sqlite3
from bs4 import BeautifulSoup
from datetime import datetime
from fastapi import FastAPI, HTTPException, UploadFile, File, Query
from fastapi.middleware.cors import CORSMiddleware
import requests
import os
import subprocess
import json
from typing import Dict, Any, List

In [6]:
# Path to dataset
import pandas as pd
html_folder = r"G:\001 IITM DATASCIENCE\002 Diploma\TDS\roe\mock\mock_roe_4"
db_path = os.path.join(html_folder, "violations.db")

# Step 1: Extract business_id for postal code 94110 from HTML files
business_ids = []

for filename in os.listdir(html_folder):
    if filename.startswith("biz-") and filename.endswith(".html"):
        file_path = os.path.join(html_folder, filename)

        with open(file_path, "r", encoding="utf-8") as file:
            soup = BeautifulSoup(file, "html.parser")

            # Loop through all tables
            for table in soup.find_all("table"):
                rows = table.find_all("tr")

                business_id = None
                postal_code = None
                latitude = None
                longitude = None

                for row in rows:
                    cols = row.find_all("td")
                    if len(cols) >= 2:
                        for i in range(len(cols) - 1):
                            if cols[i].text.strip() == "business_id":
                                business_id = cols[i + 1].text.strip()
                            if cols[i].text.strip() == "postal_code":
                                postal_code = cols[i + 1].text.strip()
                            if cols[i].text.strip() == "latitude":
                                latitude = cols[i + 1].text.strip()
                            if cols[i].text.strip() == "longitude":
                                longitude = cols[i + 1].text.strip()
                            if business_id and postal_code and latitude != "null" and latitude is not None and longitude != "null" and longitude is not None:
                                business_ids.append([business_id, postal_code, float(latitude), float(longitude)]) 

print(business_ids[0])
df_business_ids = pd.DataFrame(business_ids, columns=["business_id", "postal_code", "latitude","longitude"])

['64660', '92672', 37.77, -122.448]


In [7]:
import os
import pdfplumber
import sqlite3
import pandas as pd
from datetime import datetime

In [8]:
inspection_data = []
pdf_folder =  r"G:\001 IITM DATASCIENCE\002 Diploma\TDS\roe\mock\mock_roe_4"
for filename in os.listdir(pdf_folder):
    if filename.startswith("inspections-") and filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder, filename)

        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                table = page.extract_table()
                if table:
                    for row in table[1:]:  # Skip header row
                        try:
                            business_id = row[0].strip()  # First column: business_id
                            date_str = row[1].strip()  # Second column: date
                            score = row[2].strip()  # Third column: score
                            type_table = row[3].strip()  # Fourth column: type

                            # Convert to datetime for proper comparison
                            date_obj = datetime.strptime(date_str, "%Y-%m-%d")

                            # Only keep records on or after the cutoff date
                            if score.isdigit():
                                inspection_data.append([business_id, date_str, score, type_table])

                        except (IndexError, AttributeError, ValueError):
                            continue  # Skip invalid rows

print(inspection_data[0])
df_inspections = pd.DataFrame(inspection_data, columns=["business_id", "date", "score","type_table"])

['54', '2013-10-08', '96', 'Routine - Unscheduled']


In [9]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

query = """
    SELECT business_id, date, description 
    FROM violations 
    WHERE description LIKE '%water%' 
        OR description LIKE '%unapproved%' 
        OR description LIKE '%moderate%' 
        OR description LIKE '%facilities%' 
        OR description LIKE '%unsanitary%'
"""

df_violations = pd.read_sql_query(query, conn)
conn.close()

print(df_violations.head())

   business_id        date                                        description
0           10  2014-07-29            Insufficient hot water or running water
1           10  2014-07-29   Unapproved or unmaintained equipment or utensils
2           10  2014-01-14  Inadequate and inaccessible handwashing facili...
3           10  2016-05-03   Unapproved or unmaintained equipment or utensils
4           24  2014-11-24  Inadequate and inaccessible handwashing facili...


In [10]:
print(df_violations.head())  
print(df_inspections.head())
print(df_business_ids.head())

   business_id        date                                        description
0           10  2014-07-29            Insufficient hot water or running water
1           10  2014-07-29   Unapproved or unmaintained equipment or utensils
2           10  2014-01-14  Inadequate and inaccessible handwashing facili...
3           10  2016-05-03   Unapproved or unmaintained equipment or utensils
4           24  2014-11-24  Inadequate and inaccessible handwashing facili...
  business_id        date score             type_table
0          54  2013-10-08    96  Routine - Unscheduled
1          56  2013-10-21    94  Routine - Unscheduled
2          61  2013-10-28    92  Routine - Unscheduled
3         101  2013-10-30   100  Routine - Unscheduled
4         180  2013-10-21    91  Routine - Unscheduled
  business_id postal_code  latitude  longitude
0       64660       92672     37.77   -122.448
1       64660       92672     37.77   -122.448
2       64660       92672     37.77   -122.448
3       64660 

In [11]:
df_violations['business_id'] = df_violations['business_id'].astype(str)
df_inspections['business_id'] = df_inspections['business_id'].astype(str)
df_business_ids['business_id'] = df_business_ids['business_id'].astype(str)

In [12]:
import pandas as pd

# Assuming you have the data loaded as df_business_ids, df_inspections, df_violations

# 1. Merge df_violations with df_inspections on business_id and date
merged_df = pd.merge(df_violations, df_inspections, on=["business_id", "date"], how="inner")

# 2. Merge the result with df_business_ids on business_id
final_df = pd.merge(merged_df, df_business_ids, on="business_id", how="inner")

# Display the merged result
print(final_df.head())


  business_id        date                                        description  \
0          10  2014-07-29            Insufficient hot water or running water   
1          10  2014-07-29   Unapproved or unmaintained equipment or utensils   
2          10  2014-01-14  Inadequate and inaccessible handwashing facili...   
3          10  2016-05-03   Unapproved or unmaintained equipment or utensils   
4          24  2014-11-24  Inadequate and inaccessible handwashing facili...   

  score             type_table postal_code  latitude  longitude  
0    94  Routine - Unscheduled       94104   37.7911   -122.404  
1    94  Routine - Unscheduled       94104   37.7911   -122.404  
2    92  Routine - Unscheduled       94104   37.7911   -122.404  
3    82  Routine - Unscheduled       94104   37.7911   -122.404  
4    96  Routine - Unscheduled       94104   37.7929   -122.403  


In [13]:
import pandas as pd
# Ensure the score column is numeric, coercing errors to NaN
final_df['score'] = pd.to_numeric(final_df['score'], errors='coerce')

In [16]:
# Drop rows where score is NaN after conversion
final_df = final_df.dropna(subset=['score'])

# List of words to filter for in the description
keywords = ['water', 'unapproved', 'moderate', 'facilities', 'unsanitary']

# Create a regular expression pattern to match any of the keywords
pattern = '|'.join(keywords)

# Filter rows where description contains any of the keywords and score is 80 or more
filtered_data = final_df[
    final_df['description'].str.contains(pattern, case=False, na=False) &
    (final_df['score'] >= 80)
]

# Display the filtered result
print(filtered_data.head())

  business_id        date                                        description  \
0          10  2014-07-29            Insufficient hot water or running water   
1          10  2014-07-29   Unapproved or unmaintained equipment or utensils   
2          10  2014-01-14  Inadequate and inaccessible handwashing facili...   
3          10  2016-05-03   Unapproved or unmaintained equipment or utensils   
4          24  2014-11-24  Inadequate and inaccessible handwashing facili...   

   score             type_table postal_code  latitude  longitude  
0     94  Routine - Unscheduled       94104   37.7911   -122.404  
1     94  Routine - Unscheduled       94104   37.7911   -122.404  
2     92  Routine - Unscheduled       94104   37.7911   -122.404  
3     82  Routine - Unscheduled       94104   37.7911   -122.404  
4     96  Routine - Unscheduled       94104   37.7929   -122.403  


In [18]:
filtered_data_94110 = filtered_data[filtered_data['postal_code'] == '94110']


In [25]:
print(filtered_data_94110.dropna(inplace=True))
unique_businesses = filtered_data_94110['business_id'].nunique()
unique_businesses

None


C:\Users\ksrag\AppData\Local\Temp\ipykernel_18112\994532021.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  print(filtered_data_94110.dropna(inplace=True))


217